In [2]:
from pyalgotrade import strategy
from pyalgotrade.technical import ma
from pyalgotrade import strategy
from pyalgotrade.barfeed import csvfeed
from pyalgotrade.bar import Frequency
from pyalgotrade import plotter
import numpy as np
from pyalgotrade.barfeed import quandlfeed
import datetime
from pyalgotrade.technical import ma
filename = "nifty15min.csv"
import pytz
firstDate = datetime.datetime(2012,5,30,9,15,0,0,pytz.utc)
endDate = datetime.datetime(2016,12,9,15,29,0,0,pytz.utc)

In [3]:
def safe_round(value, digits):
    if value is not None:
        value = round(value, digits)
    return value

In [4]:
from pyalgotrade import strategy
from pyalgotrade.technical import ma
from pyalgotrade.technical import cross

In [5]:
class SMACrossOver(strategy.BacktestingStrategy):
    def __init__(self, feed, instrument):
        strategy.BacktestingStrategy.__init__(self, feed, 100000)
        self.__instrument = instrument
        self.__position = None
        # We'll use adjusted close values instead of regular close values.
        #self.setUseAdjustedValues(False)
        #self.__prices = feed[instrument].getPriceDataSeries()
        #self.__sma = ma.SMA(self.__prices, smaPeriod)
        self.__sma10 = ma.SMA(feed[instrument].getCloseDataSeries(), 10)
        self.__sma20 = ma.SMA(feed[instrument].getCloseDataSeries(), 20)

    def getSMA(self):
        return self.__sma10
    
    def getSMA(self):
        return self.__sma20
    
    def onEnterOk(self, position):
        execInfo = position.getEntryOrder().getExecutionInfo()
        self.info("BUY at %.2f" % (execInfo.getPrice()))
        
    

    def onEnterCanceled(self, position):
        self.__position = None

    def onExitOk(self, position):
        execInfo = position.getExitOrder().getExecutionInfo()
        self.info("SELL at %.2f" % (execInfo.getPrice()))
        self.__position = None
        
        

    def onExitCanceled(self, position):
        # If the exit was canceled, re-submit it.
        self.__position.exitMarket()

    def onBars(self, bars):
        # If a position was not opened, check if we should enter a long position.
        bar = bars[self.__instrument]
        if self.__position is None :
            if cross.cross_above(self.__sma10,self.__sma20) > 0 and (bar.getDateTime().hour>10 and bar.getDateTime().hour<14):
                shares = int(self.getBroker().getCash() * 0.9 / bars[self.__instrument].getPrice())
                # Enter a buy market order. The order is good till canceled.
                self.__position = self.enterLong(self.__instrument, shares, True)
        # Check if we have to exit the position.
        elif not self.__position.exitActive() and cross.cross_below(self.__sma10,self.__sma20) > 0:
            self.__position.exitMarket()

In [6]:
def run_strategy():
    # Load the bar feed from the CSV file
    #feed = quandlfeed.Feed()
    #feed.addBarsFromCSV("yes_bank", "yes_bankk.csv")

    # Evaluate the strategy with the feed.
    
    feed = csvfeed.GenericBarFeed(Frequency.MINUTE,pytz.utc)
#feed = quandlfeed.Feed()
    feed.setBarFilter(csvfeed.DateRangeFilter(firstDate,endDate))
    feed.addBarsFromCSV('bank_nifty',filename)

    #myStrategy = MyStrategy(feed, "bank_nifty")
    #myStrategy.run()
    
    crossover = SMACrossOver(feed,"bank_nifty")
    #plt = plotter.StrategyPlotter(SMACrossOver)
    crossover.run()
    #plt.plot()
    print("Final portfolio value: %.2f" % (crossover.getBroker().getEquity()))

In [7]:
run_strategy()

2012-05-31 14:00:00+00:00 strategy [INFO] BUY at 9440.00
2012-06-01 10:45:00+00:00 strategy [INFO] SELL at 9404.55
2012-06-04 13:45:00+00:00 strategy [INFO] BUY at 9222.35
2012-06-05 14:15:00+00:00 strategy [INFO] SELL at 9333.60
2012-06-12 12:45:00+00:00 strategy [INFO] BUY at 10033.00
2012-06-14 09:45:00+00:00 strategy [INFO] SELL at 10046.10
2012-06-15 11:15:00+00:00 strategy [INFO] BUY at 9908.30
2012-06-18 12:00:00+00:00 strategy [INFO] SELL at 9762.05
2012-06-19 13:00:00+00:00 strategy [INFO] BUY at 9762.00
2012-06-20 13:30:00+00:00 strategy [INFO] SELL at 9765.00
2012-06-21 12:30:00+00:00 strategy [INFO] BUY at 9831.00
2012-06-22 12:30:00+00:00 strategy [INFO] SELL at 9921.00
2012-06-26 12:45:00+00:00 strategy [INFO] BUY at 9966.00
2012-06-26 15:15:00+00:00 strategy [INFO] SELL at 9966.00
2012-06-28 11:30:00+00:00 strategy [INFO] BUY at 10032.00
2012-06-28 13:30:00+00:00 strategy [INFO] SELL at 9983.00
2012-07-04 11:30:00+00:00 strategy [INFO] BUY at 10511.00
2012-07-04 13:45:00

2013-04-30 11:30:00+00:00 strategy [INFO] SELL at 12465.65
2013-05-06 13:30:00+00:00 strategy [INFO] BUY at 12255.00
2013-05-06 15:00:00+00:00 strategy [INFO] SELL at 12298.95
2013-05-09 13:00:00+00:00 strategy [INFO] BUY at 12628.00
2013-05-10 10:00:00+00:00 strategy [INFO] SELL at 12601.65
2013-05-10 13:00:00+00:00 strategy [INFO] BUY at 12672.00
2013-05-13 13:30:00+00:00 strategy [INFO] SELL at 12642.00
2013-05-14 12:15:00+00:00 strategy [INFO] BUY at 12598.00
2013-05-14 14:30:00+00:00 strategy [INFO] SELL at 12530.00
2013-05-17 13:00:00+00:00 strategy [INFO] BUY at 13185.00
2013-05-20 12:45:00+00:00 strategy [INFO] SELL at 13208.65
2013-05-21 14:00:00+00:00 strategy [INFO] BUY at 13216.55
2013-05-22 10:15:00+00:00 strategy [INFO] SELL at 13048.00
2013-05-24 11:45:00+00:00 strategy [INFO] BUY at 12675.00
2013-05-27 14:00:00+00:00 strategy [INFO] SELL at 12770.00
2013-05-28 14:00:00+00:00 strategy [INFO] BUY at 12959.40
2013-05-29 10:00:00+00:00 strategy [INFO] SELL at 12786.80
2013-

2014-04-29 13:45:00+00:00 strategy [INFO] BUY at 13068.00
2014-04-30 09:15:00+00:00 strategy [INFO] SELL at 13022.35
2014-05-02 11:15:00+00:00 strategy [INFO] BUY at 13031.25
2014-05-02 13:45:00+00:00 strategy [INFO] SELL at 12921.55
2014-05-05 11:45:00+00:00 strategy [INFO] BUY at 12963.40
2014-05-06 10:15:00+00:00 strategy [INFO] SELL at 13069.00
2014-05-07 12:00:00+00:00 strategy [INFO] BUY at 13078.00
2014-05-08 09:15:00+00:00 strategy [INFO] SELL at 13078.00
2014-05-19 11:15:00+00:00 strategy [INFO] BUY at 15202.25
2014-05-20 10:30:00+00:00 strategy [INFO] SELL at 15262.00
2014-05-27 14:00:00+00:00 strategy [INFO] BUY at 15076.95
2014-05-29 13:15:00+00:00 strategy [INFO] SELL at 15159.60
2014-06-02 11:45:00+00:00 strategy [INFO] BUY at 14966.00
2014-06-03 12:00:00+00:00 strategy [INFO] SELL at 15247.05
2014-06-03 12:15:00+00:00 strategy [INFO] BUY at 15303.30
2014-06-03 15:15:00+00:00 strategy [INFO] SELL at 15217.95
2014-06-05 12:45:00+00:00 strategy [INFO] BUY at 15356.95
2014-0

2015-04-30 13:30:00+00:00 strategy [INFO] SELL at 18229.95
2015-05-13 14:00:00+00:00 strategy [INFO] BUY at 18102.20
2015-05-14 11:15:00+00:00 strategy [INFO] SELL at 17995.90
2015-05-21 14:00:00+00:00 strategy [INFO] BUY at 18514.95
2015-05-22 14:30:00+00:00 strategy [INFO] SELL at 18476.10
2015-05-25 13:15:00+00:00 strategy [INFO] BUY at 18350.00
2015-05-25 13:30:00+00:00 strategy [INFO] SELL at 18300.00
2015-05-26 13:15:00+00:00 strategy [INFO] BUY at 18246.00
2015-05-26 14:00:00+00:00 strategy [INFO] SELL at 18208.00
2015-06-05 14:00:00+00:00 strategy [INFO] BUY at 17665.00
2015-06-05 14:45:00+00:00 strategy [INFO] SELL at 17705.95
2015-06-09 13:00:00+00:00 strategy [INFO] BUY at 17490.00
2015-06-10 14:15:00+00:00 strategy [INFO] SELL at 17664.95
2015-06-12 12:30:00+00:00 strategy [INFO] BUY at 17363.85
2015-06-15 12:15:00+00:00 strategy [INFO] SELL at 17425.55
2015-06-16 13:45:00+00:00 strategy [INFO] BUY at 17336.00
2015-06-17 13:30:00+00:00 strategy [INFO] SELL at 17522.00
2015-

2016-04-29 11:15:00+00:00 strategy [INFO] BUY at 16845.80
2016-04-29 13:30:00+00:00 strategy [INFO] SELL at 16652.25
2016-05-04 12:45:00+00:00 strategy [INFO] BUY at 16499.00
2016-05-04 14:45:00+00:00 strategy [INFO] SELL at 16383.95
2016-05-05 12:30:00+00:00 strategy [INFO] BUY at 16409.00
2016-05-05 13:30:00+00:00 strategy [INFO] SELL at 16296.25
2016-05-06 13:45:00+00:00 strategy [INFO] BUY at 16380.05
2016-05-10 12:00:00+00:00 strategy [INFO] SELL at 16733.00
2016-05-10 13:15:00+00:00 strategy [INFO] BUY at 16814.60
2016-05-11 11:00:00+00:00 strategy [INFO] SELL at 16808.85
2016-05-11 12:45:00+00:00 strategy [INFO] BUY at 16796.15
2016-05-11 14:30:00+00:00 strategy [INFO] SELL at 16776.20
2016-05-18 13:30:00+00:00 strategy [INFO] BUY at 16723.90
2016-05-19 10:30:00+00:00 strategy [INFO] SELL at 16678.65
2016-05-20 12:45:00+00:00 strategy [INFO] BUY at 16633.10
2016-05-20 14:45:00+00:00 strategy [INFO] SELL at 16483.05
2016-05-23 12:00:00+00:00 strategy [INFO] BUY at 16523.95
2016-0

Final portfolio value: 94046.91
